# Delegation Unit Test
Performing verification of the delegation mechanism by comparing the event data to the results of cadCAD model.

In [6]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *

# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# print(module_path)
# sys.path.append(module_path)
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')


Configure environment

In [8]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [9]:
delegation_events_dict = delegation_events[0]
print("EXPECTED TRUTH FROM DELEGATION EVENTS:")
for timestep, delegation_events in delegation_events_dict.items():
    for delegation_event in delegation_events:        
        print(f"{timestep}, {delegation_event['delegator']}, {delegation_event['shares']}")
    


EXPECTED TRUTH FROM DELEGATION EVENTS:
1, 0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea, 165851.137200000000000000
1, 0xd776a7306ee6a060cebb46b46d305e88fd39ba84, 251819.644650000000000000
1, 0x53c0fa070e310ce1690b3c83b2ab7f20d74d4922, 6965.000000000000000000
1, 0x9327775e225a703125167eb423d246915bdc37e7, 9950.000000000000000000
1, 0x32345932bc863f54ef87e2ece3d458f1789b1bfc, 1991.758662500000000000
1, 0xb300749c08cab1c9ce67f66a18ea5926360ba36a, 503.905083650000000000
1, 0x57b4a5f470c85d4281f7baf5162b59243a3491df, 65257.075000000000000000
1, 0xb7797cb7b4a031c4c1e04ea14824fa5d8d30beed, 39800.000000000000000000
1, 0xf5376e847efa1ea889bfcb03706f414dade0e82c, 12488.567623993900000000
1, 0x1853bf6a892abd949aed052bb634ab96afaf31dd, 14169.714085664846265000
1, 0x1beda87a78e838687668ce2c564f1746dce7396e, 765384.845000000000000000
1, 0x7eb692f22af33d637aab31f9277653235682ebe6, 4002.893379514657642500
1, 0xb4ecbb9d116a2b38cee4ede75b0a96746a175efb, 9950.000000000000000000
1, 0x4e43a0cd79a9324645167e70

In [10]:
print("MODELED RESULTS")
delegation_event_shares = {}
for timestep, delegation_events in delegation_events_dict.items():
    for delegation_event in delegation_events:

        delegation_event_shares[timestep] = df.iloc[timestep-1].delegators[delegation_event['delegator']].shares
        print(f"{timestep}, {delegation_event['delegator']}, {delegation_event_shares[timestep]}")


MODELED RESULTS


KeyError: '0x9fe0a203e190d87cef6b4b80b24fae73f243b3ea'

In [ ]:
print("UNITTEST RESULTS")
for timestep, delegation_events in delegation_events_dict.items():
    for delegation_event in delegation_events:
        delegation_event_shares[timestep] = df.iloc[timestep-1].delegators[delegation_event['delegator']].shares
        print(f"Event: Timestep={timestep}, Delegator ID={delegation_event['delegator']}, Shares Granted={delegation_event['shares']}")
        print(f"Model: Timestep={timestep}, Delegator ID={delegation_event['delegator']}, Shares Granted={delegation_event_shares[timestep]}")
        print("Shares Equal?", delegation_event['shares'] == delegation_event_shares[timestep])
        print("Shares Within 1%?", 0.99 <= abs(delegation_event['shares'] / delegation_event_shares[timestep]) <= 1.01)
        print()


# Conclusion
The number of shares granted by delegation in the first two events match the output of the cadCAD model exactly and then the next 3 are extremely close, diverging gradually.